In [1]:
import optuna

C:\Users\81393\miniconda3\envs\market-making\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
optuna.create_study(study_name="name",direction="maximize", sampler=optuna.samplers.TPESampler(seed=42),storage="sqlite:///mm.db")

[I 2025-10-14 21:47:35,013] A new study created in RDB with name: name


In [26]:
optuna.get_all_study_summaries( storage="sqlite:///mm.db")[0].best_trial
